In [1]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"

In [1]:
import netket as nk
from netket.operator.spin import sigmax,sigmaz
import jax

N = 20
hi = nk.hilbert.Spin(s=1 / 2, N =  N)

H = -1*sigmaz(hi,0)*sigmax(hi, 1)
H += -1*sigmax(hi,N-3)*sigmaz(hi, N-2)*sigmaz(hi, N-1)
H += -1*sigmax(hi,N-2)*sigmax(hi, N-1)
H += sum([-1*sigmax(hi,i)*sigmaz(hi,(i+1))*sigmax(hi,i+2) for i in range(N-3)])
evals = nk.exact.lanczos_ed(H,  compute_eigenvectors=True)
print(evals[0])

[-20.]


In [2]:
print(evals[0].dtype)

float64


In [16]:
import netket as nk
from netket.operator.spin import sigmax,sigmaz
import jax
import time
jax.config.update("jax_enable_x64", False)
L = 24
g = nk.graph.Hypercube(length=L, n_dim=1, pbc=True)
hi = nk.hilbert.Spin(s=0.5, total_sz=0, N=g.n_nodes)
ha = nk.operator.Heisenberg(hilbert=hi, graph=g)
# compute the ground-state energy (here we only need the lowest energy, and do not need the eigenstate)
t = time.time()
evals = nk.exact.lanczos_ed(ha, compute_eigenvectors=True)
print(time.time()-t)
exact_gs_energy = evals[0]
print('The exact ground-state energy is E0=',exact_gs_energy)

33.0774130821228
The exact ground-state energy is E0= [-42.680016]


In [7]:
import netket as nk
from netket.operator.spin import sigmax,sigmaz
import jax
jax.config.update("jax_enable_x64", False)
N = 24
hi = nk.hilbert.Spin(s=1 / 2, N =  N)

H = -1*sigmaz(hi,0)*sigmax(hi, 1)
H += -1*sigmax(hi,N-3)*sigmaz(hi, N-2)*sigmaz(hi, N-1)
H += -1*sigmax(hi,N-2)*sigmax(hi, N-1)
H += sum([-1*sigmax(hi,i)*sigmaz(hi,(i+1))*sigmax(hi,i+2) for i in range(N-3)])


In [28]:
HX = -1 * sigmax(hi, 0) * sigmaz(hi, 1)
HX += -1 * sigmaz(hi, N - 3) * sigmax(hi, N - 2) * sigmax(hi, N - 1)
HX += -1 * sigmaz(hi, N - 2) * sigmaz(hi, N - 1)
HX += sum([-1 * sigmaz(hi, i) * sigmax(hi, (i + 1)) * sigmaz(hi, i + 2) for i in range(N - 3)])


In [17]:
evals = nk.exact.lanczos_ed(H,  compute_eigenvectors=True)

MemoryError: Allocation failed (probably too large).

In [18]:
import numpy as np
evals[1].dtype

dtype('float64')

In [9]:
graph=nk.graph.Chain(length=N, pbc=False)
g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
print(graph.translation_group())

PermutationGroup(elems=[Id()], degree=16)


In [8]:
import time
ma = nk.models.RBM(alpha=2)
sa = nk.sampler.MetropolisLocal(hilbert=hi)

# Optimizer
op = nk.optimizer.Sgd(learning_rate=0.05)
# Stochastic Reconfiguration
sr = nk.optimizer.SR(diag_shift=0.1)

# The variational state
vs = nk.vqs.MCState(sa, ma, n_samples=2048)

# The ground-state optimization loop
gs = nk.VMC(
    hamiltonian=H,
    optimizer=op,
    preconditioner=sr,
    variational_state=vs)

start = time.time()
gs.run(out='RBM', n_iter=25000)
end = time.time()

print('### RBM calculation')
print('Has',vs.n_parameters,'parameters')
print('The RBM calculation took',end-start,'seconds')

  2%|▏         | 523/25000 [00:39<30:47, 13.25it/s, Energy=-14.655 ± 0.052 [σ²=4.595, R̂=1.0065]] 


KeyboardInterrupt: 

In [3]:
import netket.nn as nknn
import flax.linen as nn
import jax.numpy as jnp
import numpy as np
import netket as nk

class FFNN(nn.Module):
    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=2*x.shape[-1],
                     use_bias=True,
                     param_dtype=np.complex128,
                     kernel_init=nn.initializers.normal(stddev=1.),
                     bias_init=nn.initializers.normal(stddev=1.)
                    )(x)
        x = nknn.log_cosh(x)
        x = jnp.sum(x, axis=-1)
        return x

model = FFNN()
# We shall use an exchange Sampler which preserves the global magnetization (as this is a conserved quantity in the model)
sa = nk.sampler.MetropolisHamiltonian(hilbert=hi, hamiltonian=H)

# Construct the variational state
vs = nk.vqs.MCState(sa, model, n_samples=4096)

# We choose a basic, albeit important, Optimizer: the Stochastic Gradient Descent
opt = nk.optimizer.Adam(learning_rate=0.0002)

# Stochastic Reconfiguration
sr = nk.optimizer.SR(diag_shift=0.01)

# We can then specify a Variational Monte Carlo object, using the Hamiltonian, sampler and optimizers chosen.
# Note that we also specify the method to learn the parameters of the wave-function: here we choose the efficient
# Stochastic reconfiguration (Sr), here in an iterative setup
gs = nk.VMC(hamiltonian=H, optimizer=opt, variational_state=vs, preconditioner=None)

NameError: name 'hi' is not defined

In [27]:
gs.run(out="test", n_iter=1000)

  0%|          | 5/1000 [00:18<1:01:01,  3.68s/it, Energy=-0.997-0.000j ± 0.062 [σ²=15.294, R̂=1.0019]]


KeyboardInterrupt: 